In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_id = 'cdy5d5yu'
run_name = "dandy-sweep-3"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        low_sts = get_idx_with_top_column_values('sts_score',n=n,ascending=True)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","pp_letter_diff", "reward","label_flip"]
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
split = 'training_step'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=5)
print_interesting_text_stats(df_d[split], n=5)


###############

random 0 

Original: this 72-minute film does have some exciting scenes , but it's a tad slow .
Original label 0
Unique paraphrases: 4
How the paraphrases change:


,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,reward,label_flip,
are are are are are are are are are are are are are,0.69010,0.20216,0.05399,23,0.00000,0,[1]
"This 72-minute film does have some exciting scenes, but it's a tad slow, but it's a tad slow, but it's a tad slow, but it's a tad slow, but it's a tad slow, but it's a tad slow",0.50614,0.38612,0.94528,-102,3.00000,0,[2]
"This 72-minute film does have some exciting scenes, but it's a tad slow, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.72641,0.16585,0.79238,-154,1.99023,0,"[3, 5]"
"This 72-minute film does have some exciting scenes, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.62614,0.26612,0.63413,-165,3.00000,0,[4]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
30866,7075,2,"this 72-minute film does have some exciting scenes , but it's a tad slow .","This 72-minute film does have some exciting scenes, but it's a tad slow, but it's a tad slow, but it's a tad slow, but it's a tad slow, but it's a tad slow, but it's a tad slow",0.892263,0.50614,0.50614,0,0,0,0.38612,0.94528,-102,0.420455,3,-0.257789,0.773368,316,782,0,16.0793,0.502477,0.25,24,16,60,16,1.02108,0.137016,0.0655043,0.00616284,0.0575195,5.9775e-05,0.071355,0.0144699,0.0412753,6.4723e-05,1.30751,0.0566958,2.1762e-05,1.46301e-06,4,4,1,14,1,4,2,54,44,1,13,2,124,-30,0,-9,0,-70,"[0, 18]","[this, .]","[2, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]","[This, , but it 's a tad slow, but it 's a tad slow, but it 's a tad slow, but it 's a tad slow, but it 's a tad slow]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]","[72-minute film does have some exciting scenes, but it 's a tad slow]",2,2,36,2,True,True,False,36



###############

random 1 

Original: the immersive powers of the giant screen and its hyper-realistic images are put to perfect use in the breathtakingly beautiful outer-space documentary space station 3d .
Original label 1
Unique paraphrases: 4
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
26686,6117,2,it's a bad thing when a movie has about as much substance as its end credits blooper reel .,"It's a bad thing when a movie has about as much as its end credits blooper reel, but but when a movie has about as its end credits blooper reel, it's a bad thing when a movie has about its its end credits, but it'",0.914868,0.63712,0.637121,0,0,0,0.27775,0.86139,-122,0.42723,3,-1.01428,3.04283,258,724,0,9.04133,0.282542,0,32,16,60,16,1.00583,0.136158,0.0648706,0.00643366,0.0565167,5.9613e-05,0.071133,0.0144604,0.0411386,6.4363e-05,1.31811,0.0664756,2.3498e-05,1.545e-06,5,4,1,19,1,2,0,71,47,1,6,0,163,-28,0,-4,0,-92,"[0, 13, 20]","[it, substance, .]","[1, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]","[It, , but but when a movie has about as its end credits blooper reel, it 's a bad thing when a movie has about its its end credits, but it']","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19]","['s a bad thing when a movie has about as much, as its end credits blooper reel]",2,3,34,3,True,True,False,34



###############

low_sts 4 

Original: it's a demented kitsch mess ( although the smeary digital video does match the muddled narrative ) , but it's savvy about celebrity and has more guts and energy than much of what will open this year .
Original label 1
Unique paraphrases: 5
How the paraphrases change:


,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,reward,label_flip,
It's a kitsch mess but it's savvy about celebrity and has more guts and energy than much of what will open this year.,0.89915,0.01588,0.90332,83,0.0000,0,[1]
"It's a demented mess kitsch kitsch ( although the smeary digital video does match the muddled narrative, but but it's savvy about celebrity and has more guts and energy than much of what will open this year, but it's savvy about celebrity and has more guts and energy than",0.82516,0.08988,0.98425,-72,1.0785,0,[2]
"It's a demented kitsch mess ( although the smeary digital video does match the muddled narrative), but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.11155,0.80348,0.64493,-50,3.0000,1,[3]
the by the the throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne throne,0.44658,0.46845,-0.08291,-192,0.0000,1,[4]
"It's a demented kitsch mess ( although the smeary digital video does match the muddled narrative, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.09761,0.81742,0.66018,-49,3.0000,1,[5]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
6907,1555,3,"it's a demented kitsch mess ( although the smeary digital video does match the muddled narrative ) , but it's savvy about celebrity and has more guts and energy than much of what will open this year .","It's a demented kitsch mess ( although the smeary digital video does match the muddled narrative), but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.915031,0.11155,0.888452,1,0,1,0.80348,0.64493,-50,0.8,3,-1.09813,3.29439,27,959,1,5.20505,0.162658,0.3125,48,16,60,16,0.97917,0.133105,0.0620807,0.00602793,0.0544646,6.0236e-05,0.0708787,0.0144388,0.0409946,5.9747e-05,1.3379,0.0759371,0.149106,0.0887903,5,4,3,36,1,6,0,157,54,1,4,0,193,-18,0,2,0,-36,"[0, 21, 22, 23, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]","[it, it 's savvy about, celebrity and has more guts and energy than much of what will open this year.]","[1, 26, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78]","[It, but, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but]","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]","['s a demented kitsch mess( although the smeary digital video does match the muddled narrative), but]",3,3,38,21,True,True,False,38


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

.           37136
the          5734
a            4094
it           2993
...          1691
i            1681
this         1381
an            967
there         821
if            743
as            558
what          553
while         486
?             461
in            400
and           397
with          378
:             375
like          353
for           348
one           341
;             330
though        321
not           311
despite       287
even          284
you           265
--            255
,             240
although      223
dtype: int64

#### INSERTIONS ####

The                                                                                                                                                                                              6027
A                                                                                                                                                                                                4283
,      

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level,data_split
14068,3234,4,"despite its old-hat set-up and predictable plot , empire still has enough moments to keep it entertaining .","Despite its old-hat set-up and predictable plot, empire still has enough moments to keep it entertaining, despite its old- old-hat set-up and predictable plot, despite its old-hat set-up and predictable plot, despite its old-hat set-up and predictable",0.909742,0.58334,0.583337,1,1,0,0.32641,0.93950,-144,0.426295,3.00,-0.534378,1.603134,257,1655,1,3.209224,0.100288,0.187500,32,16,60,16,1.005368,0.130776,0.060056,0.006132,0.052187,0.000058,0.070577,0.014438,0.040781,0.000057,1.311628,0.064095,0.149717,0.088856,5,4,2.0,16,1,4,0,86,37,1,13,0,202,-21,0,-9,0,-116,"[0, 24]","[despite, .]","[2, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]","[Despite, , despite its old- old- hat set- up and predictable plot, despite its old- hat set- up and predictable plot, despite its old- hat set- up and predictable]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]","[its old- hat set- up and predictable plot, empire still has enough moments to keep it entertaining]",2,2,37,2,True,True,False,37,training_step
51284,3234,5,"despite its old-hat set-up and predictable plot , empire still has enough moments to keep it entertaining .","Despite its old-hat set-up and predictable plot, empire still has enough moments to keep it entertaining, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.909742,0.33219,0.667812,1,0,1,0.57755,0.82713,-142,0.429719,3.00,-0.000055,0.000165,64,2330,0,1.564080,inf,0.171875,32,64,60,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2.0,16,1,4,0,86,52,1,4,0,194,-36,0,0,0,-108,"[0, 24]","[despite, .]","[2, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]","[Despite, , but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]","[its old- hat set- up and predictable plot, empire still has enough moments to keep it entertaining]",2,2,38,2,True,True,False,38,eval_train
2184,493,5,"despite its flaws , secretary stays in your head and makes you question your own firmly held positions .","Despite its flaws, secretary stays in your head and makes you question your own firmly held positions, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.870070,0.42747,0.572533,1,0,1,0.44260,0.84783,-154,0.403101,3.00,-0.002184,0.006553,294,2158,0,1